- API로 Nan값 없이 가져올 수 있도록 날짜 로직 수정 (100개 test)
- +- 2주 기준으로 종가, 거래량, 외국인/기관/개인 순매수량 갖고오는걸로

In [1]:
import pandas as pd

df = pd.read_csv('/Users/JooAnLee/final_project/db/news_2023_2025_summarized.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   news_id               14087 non-null  object
 1   wdate                 14087 non-null  object
 2   title                 14087 non-null  object
 3   article               14087 non-null  object
 4   press                 14087 non-null  object
 5   url                   14087 non-null  object
 6   image                 14087 non-null  object
 7   article_preprocessed  14087 non-null  object
 8   summary               14087 non-null  object
 9   stock_list            14087 non-null  object
 10  industry_list         14087 non-null  object
 11  summary_embedding     14087 non-null  object
dtypes: object(12)
memory usage: 1.3+ MB


In [2]:
pip install pykrx

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pykrx

Note: you may need to restart the kernel to use updated packages.


In [4]:
import ast

def extract_last_company_name(labels):
    # NaN 또는 빈 값 처리
    if pd.isna(labels) or labels == '[]' or labels == '':
        return None
    # 문자열인 경우 리스트로 변환
    if isinstance(labels, str):
        try:
            labels_list = ast.literal_eval(labels)
        except Exception:
            return None
    elif isinstance(labels, list):
        labels_list = labels
    else:
        return None
    # 빈 리스트 처리
    if not labels_list:
        return None
    # 마지막 값 반환
    return labels_list[-1]

# 적용
df['stock_name'] = df['stock_list'].apply(extract_last_company_name)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   news_id               14087 non-null  object
 1   wdate                 14087 non-null  object
 2   title                 14087 non-null  object
 3   article               14087 non-null  object
 4   press                 14087 non-null  object
 5   url                   14087 non-null  object
 6   image                 14087 non-null  object
 7   article_preprocessed  14087 non-null  object
 8   summary               14087 non-null  object
 9   stock_list            14087 non-null  object
 10  industry_list         14087 non-null  object
 11  summary_embedding     14087 non-null  object
 12  stock_name            14087 non-null  object
dtypes: object(13)
memory usage: 1.4+ MB


In [5]:
from pykrx import stock
from pykrx import bond
import datetime

In [6]:
from pykrx import stock

# 최근 영업일(예: 2024년 12월 31일)로 고정
recent_date = "2025-05-30"
kospi_tickers = stock.get_market_ticker_list(date=recent_date, market="KOSPI")
ticker_name_map = {stock.get_market_ticker_name(ticker): ticker for ticker in kospi_tickers}

In [7]:
def get_ticker_from_name(name):
    if pd.isna(name):
        return None
    return ticker_name_map.get(name)

df['ticker'] = df['stock_name'].apply(get_ticker_from_name)

In [8]:
df.tail()

,news_id,wdate,title,article,press,url,image,article_preprocessed,summary,stock_list,industry_list,summary_embedding,stock_name,ticker
14082,20240101_0001,2024-01-01 18:30:00,통합 셀트리온 12일 상장… 잭팟 터지나,합병 전 외국인·기관 동반 매수\n올해 영업익 1조5130억 전망\n최근 증시 상승...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://imgnews.pstatic.net/image/014/2024/01/...,새해 출범하는 '통합 셀트리온'에 대한 기대감이 커지는 것으로 풀이된다. 외국인과 ...,외국인과 기관이 지난해 12월 삼성전자 주식을 1조4267억원어치 사들이며 전체 주...,['삼성전자'],['통신 및 방송 장비 제조업'],"[0.211165189743042, -1.0113810300827026, -0.13...",삼성전자,005930
14083,20240101_0011,2024-01-01 18:15:00,"""빅딜 절호의 기회"" 새해부터 분주한 K제약바이오",'JP모건 헬스케어 컨퍼런스'\n삼바·셀트리온 등 대거 참여\n기술력 알리고 협력기...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://ssl.pstatic.net/static.news/image/news...,'JP모건 헬스케어 컨퍼런스' 삼바·셀트리온 등 대거 참여 기술력 알리고 협력기회 ...,오는 8일부터 미국 샌프란시스코에서 열리는 'JP모건 헬스케어 컨퍼런스'에 참여하는...,"['한미약품', '유한양행']","['의약품 제조업', '의약품 제조업']","[-0.2789621651172638, -0.20791760087013245, -0...",유한양행,000100
14084,20240101_0017,2024-01-01 17:45:00,"김·김치·라면 '수출 대박'…""올핸 떡볶이·핫도그가 대열 합류""",■'차세대 한류' 노리는 식품업계\n라면·김 작년 수출액 1조원 돌파\n역대급 실적...,서울경제,https://n.news.naver.com/mnews/article/011/000...,https://imgnews.pstatic.net/image/011/2024/01/...,■'차세대 한류' 노리는 식품업계 라면·김 작년 수출액 1조원 돌파 역대급 실적에 ...,"CJ 제일001040) 제일제당은 유럽, 호주 등에서 영토를 넓힘과 동시에 오카도,...",['농심'],['기타 식품 제조업'],"[-0.6008058190345764, -0.3781067132949829, -0....",농심,004370
14085,20240101_0032,2024-01-01 11:21:00,"외인·기관 홀린 '합병 셀트리온', 새해 잭팟 터트리나","[파이낸셜뉴스] 최근 증시 상승을 이끈 외국인, 기관 투자자들이 셀트리온그룹주를 순...",파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://imgnews.pstatic.net/image/014/2024/01/...,새해 출범하는 '합병 셀트리온'에 대한 기대감이 커지는 것으로 풀이된다. 외국인과 ...,외국인과 기관이 지난해 12월 삼성전자 주식을 1조4267억원어치 사들이며 전체 주...,"['삼성전자', '셀트리온']","['통신 및 방송 장비 제조업', '기초 의약물질 제조업']","[0.26362383365631104, -1.098483681678772, -0.1...",셀트리온,068270
14086,20240101_0049,2024-01-01 06:03:00,태영건설 파장 속 PF-ABCP '거래 부진'…차환 우려 고개,"PF 구조조정에 건설업 기피심리↑…""익숙한 악재라 파장 적을 것"" 관측도\n워크아웃...",연합뉴스,https://n.news.naver.com/mnews/article/001/001...,https://imgnews.pstatic.net/image/001/2024/01/...,시공능력 순위 16위의 중견기업인 태영건설이 워크아웃을 신청하면서 부동산 PF 부실...,시공능력 순위 16위의 중견기업인 태영건설이 워크아웃을 신청하면서 부동산 PF 부실...,['태영건설'],['토목 건설업'],"[-0.14925730228424072, -0.5277271866798401, -0...",태영건설,009410


In [14]:
df['news_date'] = df['wdate'].dt.normalize()

In [15]:
# wdate를 datetime 형식으로 변환 (시간 제거)
df['wdate'] = pd.to_datetime(df['wdate'])

import pandas as pd
import numpy as np


# 2023년 1월 ~ 2025년 12월까지 모든 거래일 리스트 생성
trading_days = []
for year in range(2022, 2026):
    for month in range(1, 13):
        try:
            days = stock.get_business_days(year, month)
            trading_days.extend(days)
        except Exception as e:
            # 월별 데이터가 없을 경우(미래 등) 무시
            continue
        
# trading_days는 반드시 오름차순 정렬된 pd.Series 혹은 DatetimeIndex여야 함
trading_days = pd.to_datetime(sorted(set(trading_days)))

def find_nearest_trading_day(date, trading_days):
    """date가 거래일이면 그대로, 아니면 다음 거래일 반환"""
    after = trading_days[trading_days >= date]
    if len(after) > 0:
        return after[0]
    else:
        return pd.NaT

def get_trading_day_offsets(d_day, trading_days, offsets):
    """d_day를 기준으로 trading_days에서 offset만큼 떨어진 거래일 반환"""
    result = {}
    # d_day가 trading_days에 없으면 NaT 반환
    if pd.isna(d_day) or d_day not in trading_days.values:
        for col in offsets:
            result[col] = pd.NaT
        return result
    d_idx = np.where(trading_days == d_day)[0][0]
    for col, offset in offsets.items():
        idx = d_idx + offset
        if 0 <= idx < len(trading_days):
            result[col] = trading_days[idx]
        else:
            result[col] = pd.NaT
    return result

# 1. D-day_date 생성 (wdate가 거래일이면 그대로, 아니면 다음 거래일)
df['D_day_date'] = df['news_date'].apply(lambda x: find_nearest_trading_day(x, trading_days))

# 2. offsets 정의
offsets = {
    'D_minus_14_date': -14, 'D_minus_7_date': -7, 'D_minus_3_date': -3,
    'D_minus_2_date': -2, 'D_minus_1_date': -1, 'D_day_date': 0,
    'D_plus_1_date': 1, 'D_plus_2_date': 2, 'D_plus_3_date': 3,
    'D_plus_7_date': 7, 'D_plus_14_date': 14
}
date_cols = list(offsets.keys())

# 3. 모든 기준일 채우기
def fill_all_trading_dates(row):
    d_day = row['D_day_date']
    dates = get_trading_day_offsets(d_day, trading_days, offsets)
    for col, val in dates.items():
        row[col] = val
    return row

df = df.apply(fill_all_trading_dates, axis=1)


/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_85500/844646665.py:13: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days = stock.get_business_days(year, month)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_85500/844646665.py:13: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days = stock.get_business_days(year, month)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_85500/844646665.py:13: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days = stock.get_business_days(year, month)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_85500/844646665.py:13: DeprecationWarning

In [16]:
# 모든 컬럼을 표시
pd.set_option('display.max_columns', None)

# 한 줄에 표시되는 문자 수(너무 길면 줄바꿈됨)
pd.set_option('display.width', None)

# 예시 출력
df.head()

,news_id,wdate,title,article,press,url,image,article_preprocessed,summary,stock_list,industry_list,summary_embedding,stock_name,ticker,D_day_date,D_minus_14_date,D_minus_7_date,D_minus_3_date,D_minus_2_date,D_minus_1_date,D_plus_1_date,D_plus_2_date,D_plus_3_date,D_plus_7_date,D_plus_14_date,news_date
0,20250523_0002,2025-05-23 18:52:00,"[단독] 카카오페이, 2500만 회원 쓱·스마일페이 품나…간편결제 시장 빅3 경쟁 후끈",매각가 5000억 안팎 달할듯\n결제시장 내 입지강화 포석\n카카오페이 [사진 = ...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,매각가 5000억 안팎 달할듯 결제시장 내 입지강화 포석 카카오페이 국내 대표 전...,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,['카카오페이'],['금융 지원 서비스업'],"[-0.21889664232730865, -1.4778014421463013, 0....",카카오페이,377300,2025-05-23,2025-04-30,2025-05-14,2025-05-20,2025-05-21,2025-05-22,2025-05-26,2025-05-27,2025-05-28,2025-06-04,NaT,2025-05-23
1,20250523_0004,2025-05-23 18:33:00,"골드만삭스 차기 CEO, 이재용·김병주·이창용 만났다",[사진=연합뉴스]\n세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,['삼성전자'],['통신 및 방송 장비 제조업'],"[-0.2798689603805542, -0.5640878081321716, -0....",삼성전자,005930,2025-05-23,2025-04-30,2025-05-14,2025-05-20,2025-05-21,2025-05-22,2025-05-26,2025-05-27,2025-05-28,2025-06-04,NaT,2025-05-23
2,20250523_0007,2025-05-23 18:00:00,[단독] 전자결제 강자 카카오페이 쓱·스마일 페이 인수 추진,국내 대표 전자결제사업자인 카카오페이가 신세계이마트 산하 간편결제사업부 인수에 나섰...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://ssl.pstatic.net/static.news/image/news...,국내 대표 전자결제사업자인 카카오페이가 신세계이마트 산하 간편결제사업부 인수에 나섰...,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,['카카오페이'],['금융 지원 서비스업'],"[-0.5892677903175354, -1.0036863088607788, 0.3...",카카오페이,377300,2025-05-23,2025-04-30,2025-05-14,2025-05-20,2025-05-21,2025-05-22,2025-05-26,2025-05-27,2025-05-28,2025-06-04,NaT,2025-05-23
3,20250523_0010,2025-05-23 17:52:00,조현준 효성重 지분 4.9% 美 테크펀드 2600억 매각,조현준 효성 회장이 상속세 재원 마련을 위해 효성중공업 지분 4.90%를 미국의 글...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://ssl.pstatic.net/static.news/image/news...,효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중공업 주식...,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,['효성중공업'],"['전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업']","[-0.326259970664978, -0.49666550755500793, -0....",효성중공업,298040,2025-05-23,2025-04-30,2025-05-14,2025-05-20,2025-05-21,2025-05-22,2025-05-26,2025-05-27,2025-05-28,2025-06-04,NaT,2025-05-23
4,20250523_0011,2025-05-23 17:52:00,몸집 키우는 카카오…'간편결제 빅3' 흔드나,"카카오페이, 신세계 페이 품나\n이커머스 결제 강화 점유 확대\n압도적 1위 네이버...",매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,"카카오페이, 신세계 페이 품나 이커머스 결제 강화 점유 확대 압도적 1위 네이버에 ...",카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,['카카오페이'],['금융 지원 서비스업'],"[-0.771115779876709, -1.174679160118103, -0.27...",카카오페이,377300,2025-05-23,2025-04-30,2025-05-14,2025-05-20,2025-05-21,2025-05-22,2025-05-26,2025-05-27,2025-05-28,2025-06-04,NaT,2025-05-23


In [17]:
# 6. 뉴스에 등장하는 코스피 티커만 추출
all_tickers = df['ticker'].dropna().unique().tolist()

# 7. 기준일에 등장하는 모든 날짜 추출
all_dates = pd.concat([df[col] for col in date_cols]).dropna().unique()
all_dates = sorted([d.strftime("%Y%m%d") for d in pd.to_datetime(all_dates)])


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   news_id               14087 non-null  object        
 1   wdate                 14087 non-null  datetime64[ns]
 2   title                 14087 non-null  object        
 3   article               14087 non-null  object        
 4   press                 14087 non-null  object        
 5   url                   14087 non-null  object        
 6   image                 14087 non-null  object        
 7   article_preprocessed  14087 non-null  object        
 8   summary               14087 non-null  object        
 9   stock_list            14087 non-null  object        
 10  industry_list         14087 non-null  object        
 11  summary_embedding     14087 non-null  object        
 12  stock_name            14087 non-null  object        
 13  ticker          

In [19]:
# 4. OHLCV 데이터 일괄 수집
all_tickers = df['ticker'].dropna().unique().tolist()
all_dates = pd.concat([df[col] for col in date_cols]).dropna().unique()
all_dates = sorted([d.strftime('%Y%m%d') for d in pd.to_datetime(all_dates)])

ohlcv_dict = {}
for ticker in all_tickers:
    try:
        ohlcv = stock.get_market_ohlcv_by_date(
            fromdate=min(all_dates),
            todate=max(all_dates),
            ticker=ticker
        )
        ohlcv_dict[ticker] = ohlcv
    except:
        continue

def get_ohlcv_val(ticker, date, col):
    if pd.isna(ticker) or pd.isna(date):
        return np.nan
    date_str = date.strftime('%Y%m%d')
    if ticker not in ohlcv_dict:
        return np.nan
    df_ticker = ohlcv_dict[ticker]
    if date_str not in df_ticker.index:
        return np.nan
    return df_ticker.loc[date_str, col]

In [20]:
# 5. 투자자별 순매수 데이터 일괄 수집
trading_dict = {}
for ticker in all_tickers:
    try:
        tv_df = stock.get_market_trading_value_by_date(
            fromdate=min(all_dates),
            todate=max(all_dates),
            ticker=ticker
        )
        trading_dict[ticker] = tv_df
    except:
        continue

def fast_trading_value(ticker, date, investor):
    if pd.isna(ticker) or pd.isna(date):
        return np.nan
    date_str = date.strftime('%Y%m%d')
    if ticker not in trading_dict:
        return np.nan
    tv_df = trading_dict[ticker]
    if date_str not in tv_df.index:
        return np.nan
    col_map = {
        '외국인': '외국인합계',
        '기관': '기관합계',
        '개인': '개인'
    }
    return tv_df.loc[date_str, col_map[investor]]

In [21]:
import numpy as np
from datetime import timedelta

# 6. 기준일별 컬럼 생성 (종가, 거래량, 외국인, 기관, 개인)
for col in date_cols:
    df[f'{col}_close'] = df.apply(lambda row: get_ohlcv_val(row['ticker'], row[col], '종가'), axis=1)
    df[f'{col}_volume'] = df.apply(lambda row: get_ohlcv_val(row['ticker'], row[col], '거래량'), axis=1)
    df[f'{col}_foreign'] = df.apply(lambda row: fast_trading_value(row['ticker'], row[col], '외국인'), axis=1)
    df[f'{col}_institution'] = df.apply(lambda row: fast_trading_value(row['ticker'], row[col], '기관'), axis=1)
    df[f'{col}_individual'] = df.apply(lambda row: fast_trading_value(row['ticker'], row[col], '개인'), axis=1)

# 7. D-day 등락률 계산
# D-day 시가 컬럼을 먼저 넣어야 합니다!
df['D_day_date_open'] = df.apply(lambda row: get_ohlcv_val(row['ticker'], row['D_day_date'], '시가'), axis=1)

df['D-day_change_open'] = (
    (df['D_day_date_close'] - df['D_day_date_open']) / df['D_day_date_open'] * 100
)
# 7. D-day 등락률 계산
df['D-day_change'] = (df['D_day_date_close'] - df['D_minus_1_date_close']) / df['D_minus_1_date_close'] * 100

print(df[
    ['news_id', 'stock_name', 'ticker', 'wdate', 'D_day_date_close', 'D-day_change']
    + [f'{col}_close' for col in date_cols]
].head())

         news_id stock_name  ticker               wdate  D_day_date_close  \
0  20250523_0002      카카오페이  377300 2025-05-23 18:52:00           30800.0   
1  20250523_0004       삼성전자  005930 2025-05-23 18:33:00           54200.0   
2  20250523_0007      카카오페이  377300 2025-05-23 18:00:00           30800.0   
3  20250523_0010      효성중공업  298040 2025-05-23 17:52:00          600000.0   
4  20250523_0011      카카오페이  377300 2025-05-23 17:52:00           30800.0   

   D-day_change  D_minus_14_date_close  D_minus_7_date_close  \
0      1.650165                31650.0               30200.0   
1     -0.914077                55500.0               57400.0   
2      1.650165                31650.0               30200.0   
3      1.180438               489500.0              556000.0   
4      1.650165                31650.0               30200.0   

   D_minus_3_date_close  D_minus_2_date_close  D_minus_1_date_close  \
0               29050.0               29700.0               30300.0   
1         

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 84 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   news_id                      14087 non-null  object        
 1   wdate                        14087 non-null  datetime64[ns]
 2   title                        14087 non-null  object        
 3   article                      14087 non-null  object        
 4   press                        14087 non-null  object        
 5   url                          14087 non-null  object        
 6   image                        14087 non-null  object        
 7   article_preprocessed         14087 non-null  object        
 8   summary                      14087 non-null  object        
 9   stock_list                   14087 non-null  object        
 10  industry_list                14087 non-null  object        
 11  summary_embedding            14087 non-nu

In [23]:
# 모든 컬럼을 표시
pd.set_option('display.max_columns', None)

# 한 줄에 표시되는 문자 수(너무 길면 줄바꿈됨)
pd.set_option('display.width', None)

# 예시 출력
df.head()

,news_id,wdate,title,article,press,url,image,article_preprocessed,summary,stock_list,industry_list,summary_embedding,stock_name,ticker,D_day_date,D_minus_14_date,D_minus_7_date,D_minus_3_date,D_minus_2_date,D_minus_1_date,D_plus_1_date,D_plus_2_date,D_plus_3_date,D_plus_7_date,D_plus_14_date,news_date,D_minus_14_date_close,D_minus_14_date_volume,D_minus_14_date_foreign,D_minus_14_date_institution,D_minus_14_date_individual,D_minus_7_date_close,D_minus_7_date_volume,D_minus_7_date_foreign,D_minus_7_date_institution,D_minus_7_date_individual,D_minus_3_date_close,D_minus_3_date_volume,D_minus_3_date_foreign,D_minus_3_date_institution,D_minus_3_date_individual,D_minus_2_date_close,D_minus_2_date_volume,D_minus_2_date_foreign,D_minus_2_date_institution,D_minus_2_date_individual,D_minus_1_date_close,D_minus_1_date_volume,D_minus_1_date_foreign,D_minus_1_date_institution,D_minus_1_date_individual,D_day_date_close,D_day_date_volume,D_day_date_foreign,D_day_date_institution,D_day_date_individual,D_plus_1_date_close,D_plus_1_date_volume,D_plus_1_date_foreign,D_plus_1_date_institution,D_plus_1_date_individual,D_plus_2_date_close,D_plus_2_date_volume,D_plus_2_date_foreign,D_plus_2_date_institution,D_plus_2_date_individual,D_plus_3_date_close,D_plus_3_date_volume,D_plus_3_date_foreign,D_plus_3_date_institution,D_plus_3_date_individual,D_plus_7_date_close,D_plus_7_date_volume,D_plus_7_date_foreign,D_plus_7_date_institution,D_plus_7_date_individual,D_plus_14_date_close,D_plus_14_date_volume,D_plus_14_date_foreign,D_plus_14_date_institution,D_plus_14_date_individual,D_day_date_open,D-day_change_open,D-day_change
0,20250523_0002,2025-05-23 18:52:00,"[단독] 카카오페이, 2500만 회원 쓱·스마일페이 품나…간편결제 시장 빅3 경쟁 후끈",매각가 5000억 안팎 달할듯\n결제시장 내 입지강화 포석\n카카오페이 [사진 = ...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,매각가 5000억 안팎 달할듯 결제시장 내 입지강화 포석 카카오페이 국내 대표 전...,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,['카카오페이'],['금융 지원 서비스업'],"[-0.21889664232730865, -1.4778014421463013, 0....",카카오페이,377300,2025-05-23,2025-04-30,2025-05-14,2025-05-20,2025-05-21,2025-05-22,2025-05-26,2025-05-27,2025-05-28,2025-06-04,NaT,2025-05-23,31650.0,265196.0,1.007898e+09,9.504541e+08,-2.184160e+09,30200.0,279765.0,6.984684e+08,3.078252e+09,-3.929992e+09,29050.0,120186.0,-1.122965e+09,2.494400e+06,1.171825e+09,29700.0,176027.0,3.527352e+08,1.214762e+09,-1.567497e+09,30300.0,197238.0,5.748004e+08,1.166289e+09,-1.916639e+09,30800.0,137901.0,1.122348e+09,1.742804e+08,-1.333716e+09,32100.0,282041.0,4.607171e+09,1.323014e+09,-5.921785e+09,35950.0,1389212.0,1.470187e+10,8.992374e+09,-2.367846e+10,38900.0,1376243.0,1.664487e+10,5.682036e+09,-2.231952e+10,37300.0,366439.0,3.214605e+09,-1.086590e+09,-1.934594e+09,NaN,NaN,NaN,NaN,NaN,30000.0,2.666667,1.650165
1,20250523_0004,2025-05-23 18:33:00,"골드만삭스 차기 CEO, 이재용·김병주·이창용 만났다",[사진=연합뉴스]\n세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,['삼성전자'],['통신 및 방송 장비 제조업'],"[-0.2798689603805542, -0.5640878081321716, -0....",삼성전자,005930,2025-05-23,2025-04-30,2025-05-14,2025-05-20,2025-05-21,2025-05-22,2025-05-26,2025-05-27,2025-05-28,2025-06-04,NaT,2025-05-23,55500.0,13014189.0,-9.023327e+10,-3.725153e+10,9.720042e+10,57400.0,12468089.0,-9.591462e+09,7.082969e+10,-5.880701e+10,55900.0,9080577.0,-3.937972e+10,5.456993e+09,3.462583e+10,55700.0,7794181.0,-4.721363e+10,-1.875936e+10,6.405338e+10,54700.0,15254278.0,-1.731367e+11,-1.137931e+11,2.723496e+11,54200.0,11247115.0,-1.425082e+11,-5.334769e+10,1.864062e+11,54700.0,10901337.0,-1.130642e+11,5.818121e+10,4.709796e+10,53900.0,13439520.0,-2.698350e+11,2.269069e+10,2.290161e+11,55900.0,17516283.0,1.181328e+11,1.940424e+11,-2.694841e+11,57800.0,19649983.0,1.197751e+11,6.706483e+10,-1.762907e+11,NaN,NaN,NaN,NaN,NaN,55000.0,-1.454545,-0.914077
2,20250523_0007,2025

In [25]:
df.tail()

,news_id,wdate,title,article,press,url,image,article_preprocessed,summary,stock_list,industry_list,summary_embedding,stock_name,ticker,D_day_date,D_minus_14_date,D_minus_7_date,D_minus_3_date,D_minus_2_date,D_minus_1_date,D_plus_1_date,D_plus_2_date,D_plus_3_date,D_plus_7_date,D_plus_14_date,news_date,D_minus_14_date_close,D_minus_14_date_volume,D_minus_14_date_foreign,D_minus_14_date_institution,D_minus_14_date_individual,D_minus_7_date_close,D_minus_7_date_volume,D_minus_7_date_foreign,D_minus_7_date_institution,D_minus_7_date_individual,D_minus_3_date_close,D_minus_3_date_volume,D_minus_3_date_foreign,D_minus_3_date_institution,D_minus_3_date_individual,D_minus_2_date_close,D_minus_2_date_volume,D_minus_2_date_foreign,D_minus_2_date_institution,D_minus_2_date_individual,D_minus_1_date_close,D_minus_1_date_volume,D_minus_1_date_foreign,D_minus_1_date_institution,D_minus_1_date_individual,D_day_date_close,D_day_date_volume,D_day_date_foreign,D_day_date_institution,D_day_date_individual,D_plus_1_date_close,D_plus_1_date_volume,D_plus_1_date_foreign,D_plus_1_date_institution,D_plus_1_date_individual,D_plus_2_date_close,D_plus_2_date_volume,D_plus_2_date_foreign,D_plus_2_date_institution,D_plus_2_date_individual,D_plus_3_date_close,D_plus_3_date_volume,D_plus_3_date_foreign,D_plus_3_date_institution,D_plus_3_date_individual,D_plus_7_date_close,D_plus_7_date_volume,D_plus_7_date_foreign,D_plus_7_date_institution,D_plus_7_date_individual,D_plus_14_date_close,D_plus_14_date_volume,D_plus_14_date_foreign,D_plus_14_date_institution,D_plus_14_date_individual,D_day_date_open,D-day_change_open,D-day_change
14082,20240101_0001,2024-01-01 18:30:00,통합 셀트리온 12일 상장… 잭팟 터지나,합병 전 외국인·기관 동반 매수\n올해 영업익 1조5130억 전망\n최근 증시 상승...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://imgnews.pstatic.net/image/014/2024/01/...,새해 출범하는 '통합 셀트리온'에 대한 기대감이 커지는 것으로 풀이된다. 외국인과 ...,외국인과 기관이 지난해 12월 삼성전자 주식을 1조4267억원어치 사들이며 전체 주...,['삼성전자'],['통신 및 방송 장비 제조업'],"[0.211165189743042, -1.0113810300827026, -0.13...",삼성전자,005930,2024-01-02,2023-12-08,2023-12-19,2023-12-26,2023-12-27,2023-12-28,2024-01-03,2024-01-04,2024-01-05,2024-01-11,2024-01-22,2024-01-01,72600.0,10859463.0,1.013434e+11,1.594023e+11,-2.615169e+11,73400.0,8907632.0,-3.808070e+10,8.831266e+10,-4.647848e+10,76600.0,13164909.0,3.718129e+10,3.030082e+11,-3.324287e+11,78000.0,20651042.0,3.203872e+11,-8.556869e+10,-2.286355e+11,78500.0,17797536.0,8.593716e+10,2.217516e+11,-2.453861e+11,79600.0,17142847.0,1.829740e+11,4.509347e+10,-2.259543e+11,77000.0,21753644.0,3.012775e+10,-3.028125e+11,2.788679e+11,76600.0,15324439.0,4.884858e+10,-1.909815e+11,1.414521e+11,76600.0,11304316.0,-4.129268e+10,-8.054751e+10,1.186206e+11,73200.0,57691266.0,1.686454e+12,-2.273992e+12,5.775303e+11,75100.0,19673375.0,1.372444e+11,-1.237386e+10,-1.303683e+11,78200.0,1.790281,1.401274
14083,20240101_0011,2024-01-01 18:15:00,"""빅딜 절호의 기회"" 새해부터 분주한 K제약바이오",'JP모건 헬스케어 컨퍼런스'\n삼바·셀트리온 등 대거 참여\n기술력 알리고 협력기...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://ssl.pstatic.net/static.news/image/news...,'JP모건 헬스케어 컨퍼런스' 삼바·셀트리온 등 대거 참여 기술력 알리고 협력기회 ...,오는 8일부터 미국 샌프란시스코에서 열리는 'JP모건 헬스케어 컨퍼런스'에 참여하는...,"['한미약품', '유한양행']","['의약품 제조업', '의약품 제조업']","[-0.2789621651172638, -0.20791760087013245, -0...",유한양행,000100,2024-01-02,2023-12-08,2023-12-19,2023-12-26,2023-12-27,2023-12-28,2024-01-03,2024-01-04,2024-01-05,2024-01-11,2024-01-22,2024-01-01,62975.0,320024.0,-2.842139e+09,4.934859e+09,-1.929674e+09,62593.0,252386.0,4.417311e+08,3.129644e+08,-8.031540e+08,66900.0,838339.0,-7.973058e+09,6.728543e+09,1.369021e+09,67900.0,528358.0,-2.931688e+09,7.661207e+08,2.134249e+09,68800.0,379712.0,-1.550584e+09,3.935138e+09,-2.382394e+09,67800.0,377268.0,-1.245356e+09,-3.098103e+09,4.108898e+09,68800.0,562166.0,-7.753727e+08,9.021201e+08,-2.286217e+08,66300.0,371864.0,-3.629434e+08,-4.361672e+09,4.603488e+09,65900.0,301451.0,1.226066e+09,-1.401949e+09,2.605134e+08,65300.0,265973.0,-3.847724e+08,1.253497e+09,-1.165391e+09,59600.0,2

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 86 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   news_id                      14087 non-null  object        
 1   wdate                        14087 non-null  datetime64[ns]
 2   title                        14087 non-null  object        
 3   article                      14087 non-null  object        
 4   press                        14087 non-null  object        
 5   url                          14087 non-null  object        
 6   image                        14087 non-null  object        
 7   article_preprocessed         14087 non-null  object        
 8   summary                      14087 non-null  object        
 9   stock_list                   14087 non-null  object        
 10  industry_list                14087 non-null  object        
 11  summary_embedding            14087 non-nu

In [38]:
df.drop(columns=['exchange_rate', 'bond_yield'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 84 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   news_id                      14087 non-null  object        
 1   wdate                        14087 non-null  datetime64[ns]
 2   title                        14087 non-null  object        
 3   article                      14087 non-null  object        
 4   press                        14087 non-null  object        
 5   url                          14087 non-null  object        
 6   image                        14087 non-null  object        
 7   article_preprocessed         14087 non-null  object        
 8   summary                      14087 non-null  object        
 9   stock_list                   14087 non-null  object        
 10  industry_list                14087 non-null  object        
 11  summary_embedding            14087 non-nu

- feature transform
    - 변화량/변화율 
- 기준금리, 환율, 국채 10년 수익률 데이터

## 한국은행 Ecos API - 환율, 국채 10년 수익률(금리)

In [44]:
import requests
import pandas as pd
from datetime import timedelta

# 한국은행 ECON API 키
api_key = "BFKGNG1KJWZR9DVAIN6K"
start_date = "20221010"
end_date = "20250530"

def get_bond10y(api_key, start_date, end_date):
    stat_code = "817Y002"
    item_code = "010200000"  # 국고채 10년 (실제 코드 확인 필요)
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/1000/{stat_code}/D/{start_date}/{end_date}/{item_code}/"
    response = requests.get(url)
    data = response.json()
    if "StatisticSearch" not in data or "row" not in data["StatisticSearch"]:
        print("국고채 10년 데이터 없음:", data)
        return pd.DataFrame()
    rows = data["StatisticSearch"]["row"]
    df = pd.DataFrame(rows)
    df['date'] = pd.to_datetime(df['TIME'], format='%Y%m%d')
    df['bond10y'] = pd.to_numeric(df['DATA_VALUE'], errors='coerce')
    return df[['date', 'bond10y']]

def get_fx(api_key, start_date, end_date, item_code="0000001"):
    # item_code: 0002 = 미국 달러 (주요국 통화의 대원화 환율)
    stat_code = "731Y001"
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/1000/{stat_code}/D/{start_date}/{end_date}/{item_code}/"
    response = requests.get(url)
    data = response.json()
    if "StatisticSearch" not in data or "row" not in data["StatisticSearch"]:
        print("환율 데이터 없음:", data)
        return pd.DataFrame()
    rows = data["StatisticSearch"]["row"]
    df = pd.DataFrame(rows)
    df['date'] = pd.to_datetime(df['TIME'], format='%Y%m%d')
    df['usdkrw'] = pd.to_numeric(df['DATA_VALUE'], errors='coerce')
    return df[['date', 'usdkrw']]

df_bond = get_bond10y(api_key, start_date, end_date)
df_fx = get_fx(api_key, start_date, end_date, item_code="0000001")  # 미국 달러

print(df_bond.head())
print(df_fx.head())


        date  bond10y
0 2022-10-11    4.342
1 2022-10-12    4.107
2 2022-10-13    4.203
3 2022-10-14    4.207
4 2022-10-17    4.239
        date  usdkrw
0 2022-10-11  1410.3
1 2022-10-12  1433.5
2 2022-10-13  1430.4
3 2022-10-14  1427.0
4 2022-10-17  1428.9


In [ ]:
# 1. 날짜 컬럼 타입 통일
df['news_date'] = pd.to_datetime(df['news_date'])
df_fx = df_fx.sort_values('date')
df_bond = df_bond.sort_values('date')
df = df.sort_values('news_date')

# 2. 환율 병합 (news_date에 가장 가까운 과거값)
df = pd.merge_asof(df, df_fx.rename(columns={'date': 'news_date', 'usdkrw': 'fx'}), on='news_date', direction='backward')

# 3. 국채10년 병합 (news_date에 가장 가까운 과거값)
df = pd.merge_asof(df, df_bond.rename(columns={'date': 'news_date'}), on='news_date', direction='backward')

# 4. 결과 확인
print(df[['news_date', 'fx', 'bond10y']].head())

   news_date      fx  bond10y
0 2024-01-01  1289.4    3.154
1 2024-01-01  1289.4    3.154
2 2024-01-01  1289.4    3.154
3 2024-01-01  1289.4    3.154
4 2024-01-01  1289.4    3.154


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 86 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   news_id                      14087 non-null  object        
 1   wdate                        14087 non-null  datetime64[ns]
 2   title                        14087 non-null  object        
 3   article                      14087 non-null  object        
 4   press                        14087 non-null  object        
 5   url                          14087 non-null  object        
 6   image                        14087 non-null  object        
 7   article_preprocessed         14087 non-null  object        
 8   summary                      14087 non-null  object        
 9   stock_list                   14087 non-null  object        
 10  industry_list                14087 non-null  object        
 11  summary_embedding            14087 non-nu

In [62]:
def get_rate(api_key):
    stat_code = "722Y001"  
    start_date = "202201"  
    end_date = "202505"
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/1000/{stat_code}/M/{start_date}/{end_date}/"
    response = requests.get(url)
    data = response.json()
    if "StatisticSearch" in data and "row" in data["StatisticSearch"]:
        rows = data["StatisticSearch"]["row"]
        df_rate = pd.DataFrame(rows)
        # 🔽 "한국은행 기준금리" 항목만 선택
        df_rate = df_rate[df_rate['ITEM_NAME1'] == "한국은행 기준금리"]
        if df_rate.empty:
            print("기준금리(변경일) 데이터 없음")
            return pd.DataFrame()
        df_rate['date'] = pd.to_datetime(df_rate['TIME'], format='%Y%m')
        df_rate['base_rate'] = pd.to_numeric(df_rate['DATA_VALUE'], errors='coerce')
        df_rate = df_rate.drop_duplicates(subset='date', keep='last')
        df_rate = df_rate.sort_values('date').reset_index(drop=True)
        # 날짜 범위 생성 (데이터 존재시)
        min_date = df_rate['date'].min()
        max_date = df_rate['date'].max()
        all_days = pd.DataFrame({'date': pd.date_range(min_date, max_date)})
        df_rate = df_rate.set_index('date')
        all_days['base_rate'] = df_rate['base_rate'].reindex(all_days['date'], method='ffill').values
        return all_days
    else:
        print("기준금리 데이터 없음:", data)
        return pd.DataFrame()

In [64]:
df_rate = get_rate(api_key)
print(df_rate.head())
print("----------------------------")
print(df_rate.tail(20))

        date  base_rate
0 2022-01-01       1.25
1 2022-01-02       1.25
2 2022-01-03       1.25
3 2022-01-04       1.25
4 2022-01-05       1.25
----------------------------
           date  base_rate
1167 2025-03-13       2.75
1168 2025-03-14       2.75
1169 2025-03-15       2.75
1170 2025-03-16       2.75
1171 2025-03-17       2.75
1172 2025-03-18       2.75
1173 2025-03-19       2.75
1174 2025-03-20       2.75
1175 2025-03-21       2.75
1176 2025-03-22       2.75
1177 2025-03-23       2.75
1178 2025-03-24       2.75
1179 2025-03-25       2.75
1180 2025-03-26       2.75
1181 2025-03-27       2.75
1182 2025-03-28       2.75
1183 2025-03-29       2.75
1184 2025-03-30       2.75
1185 2025-03-31       2.75
1186 2025-04-01       2.75


In [65]:
rate_df = pd.read_csv("/Users/JooAnLee/final_project/eda/jooan/korea_base_rate_daily.csv")
rate_df.head()

,date,rate
0,2008-01-01,5.0
1,2008-01-02,5.0
2,2008-01-03,5.0
3,2008-01-04,5.0
4,2008-01-05,5.0


In [66]:
rate_df['date'] = pd.to_datetime(rate_df['date'])
# 정렬 (merge_asof는 반드시 정렬 필요)
rate_df_sorted = rate_df.sort_values('date')
df_sorted = df.sort_values('news_date')

# merge_asof로 가장 가까운 과거 날짜의 base_rate 병합
merged_df = pd.merge_asof(
    df_sorted,
    rate_df_sorted.rename(columns={'date': 'news_date', 'rate': 'base_rate'}),
    on='news_date',
    direction='backward'
)

# 결과 확인
print(merged_df[['news_date', 'base_rate']].head())

   news_date  base_rate
0 2024-01-01        3.5
1 2024-01-01        3.5
2 2024-01-01        3.5
3 2024-01-01        3.5
4 2024-01-01        3.5


In [67]:
merged_df.to_csv("news(23-25)_summarized_external.csv", index=False)

In [ ]:
import pandas as pd

def get_base_rate(df, rate_csv_path, news_date_col='news_date'):
    """
    뉴스 데이터프레임(df)에 기준금리 CSV(rate_csv_path)에서 가장 가까운 과거 날짜의 base_rate를 추가합니다.
    :param df: 뉴스 데이터프레임 (news_date 컬럼 필요)
    :param rate_csv_path: 기준금리 CSV 파일 경로 (date, rate 컬럼 필요)
    :param news_date_col: 뉴스 날짜 컬럼명 (기본값: 'news_date')
    :return: base_rate 컬럼이 추가된 데이터프레임
    """
    # 기준금리 데이터 불러오기 및 전처리
    rate_df = pd.read_csv(rate_csv_path)
    rate_df['date'] = pd.to_datetime(rate_df['date'])
    rate_df = rate_df.sort_values('date')
    
    # 뉴스 날짜 컬럼 datetime 변환 및 정렬
    df = df.copy()
    df[news_date_col] = pd.to_datetime(df[news_date_col])
    df = df.sort_values(news_date_col)
    
    # merge_asof로 가장 가까운 과거 날짜의 base_rate 병합
    merged = pd.merge_asof(
        df,
        rate_df.rename(columns={'date': news_date_col, 'rate': 'base_rate'}),
        on=news_date_col,
        direction='backward'
    )
    return merged
